In [2]:
import numpy as np
import math
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

In [3]:
transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.1307,),(0.3081,)), # mean value = 0.1307, standard deviation value = 0.3081
])

In [4]:
data_path = './MNIST'

training_set = datasets.MNIST(root = data_path, train= True, download=True, transform= transform)
testing_set = datasets.MNIST(root = data_path, train= False, download=True, transform= transform)

In [7]:
from torch import nn
from torch.nn import functional as F

In [30]:
class classification(nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        
        # construct layers for a neural network
        self.classifier1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=20*20),
            nn.Sigmoid(),
        ) 
        self.classifier2 = nn.Sequential(
            nn.Linear(in_features=20*20, out_features=10*10),
            nn.Sigmoid(),
        ) 
        self.classifier3 = nn.Sequential(
            nn.Linear(in_features=10*10, out_features=10),
            nn.LogSoftmax(dim=1),
        ) 
        
        
    def forward(self, inputs):                 # [batchSize, 1, 28, 28]
        x = inputs.view(inputs.size(0), -1)    # [batchSize, 28*28]
        x = self.classifier1(x)                # [batchSize, 20*20]
        x = self.classifier2(x)                # [batchSize, 10*10]
        out = self.classifier3(x)              # [batchSize, 10]
        
        return out


In [31]:
device = torch.device('cpu')

In [32]:
criterion = nn.NLLLoss()
classifier = classification().to(device)
learning_rate_value = 0.01
optimizer = torch.optim.SGD(classifier.parameters(), lr=learning_rate_value)

In [33]:
def calc(epochs, tr_data_set, te_dataset):
    tr_loss_Liters = []
    tr_acc_Liters = []
    te_loss_Liters = []
    te_acc_Liters = []
    
    for i in range (epochs):
        tr_avg_loss = 0
        tr_avg_acc = 0
        te_avg_loss =0
        te_avg_acc = 0
        
        for x, y in tr_data_set:
            x = x.view(-1, 28*28).to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_pred = classifier(x)
            tr_loss = criterion(y_pred, y)
            tr_loss.backward()
            optimizer.step()
            
            tr_avg_loss += tr_loss / len(tr_data_set)
            pred = torch.argmax(y_pred, 1) == y
            tr_acc = pred.float().mean()
            tr_avg_acc += tr_acc.item() / len(tr_data_set)
            
        for x,y in te_dataset:
            x = x.view(-1, 28*28).to(device)
            y = y.to(device)
            y_pred = classifier(x)
            te_loss = criterion(y_pred, y)
            
            te_avg_loss += te_loss / len(te_dataset)
            pred = torch.argmax(y_pred, 1) == y
            te_acc = pred.float().mean()
            te_avg_acc += te_acc.item() / len(te_dataset)
            
        
        tr_loss_Liters.append(tr_avg_loss)
        tr_acc_Liters.append(tr_avg_acc)
        te_loss_Liters.append(te_avg_loss)
        te_acc_Liters.append(te_avg_acc)
        print(i)
        
    return tr_loss_Liters, tr_acc_Liters, te_loss_Liters, te_acc_Liters
            